# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
output_csv_file = "../WeatherPy/city_data_output.csv"

weather_df = pd.read_csv(output_csv_file)

weather_df = weather_df.drop(columns=["Unnamed: 0"])
weather_df



,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
0,bambous virieux,MU,79.05,69.0,40.0,14.97,-20.3428,57.7575
1,howard springs,AU,81.55,67.0,100.0,3.53,-12.4970,131.0470
2,port alfred,ZA,63.18,95.0,100.0,1.99,-33.5906,26.8910
3,iqaluit,CA,0.70,77.0,20.0,0.00,63.7506,-68.5145
4,busselton,AU,64.54,51.0,4.0,11.16,-33.6500,115.3333
...,...,...,...,...,...,...,...,...
626,xai-xai,MZ,78.28,68.0,3.0,6.78,-25.0519,33.6442
627,sabzevar,IR,83.48,20.0,40.0,16.11,36.2126,57.6819
628,vao,NC,74.89,75.0,79.0,20.31,-22.6667,167.4833
629,umm jarr,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
weather_df=weather_df.dropna()
weather_df


,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
0,bambous virieux,MU,79.05,69.0,40.0,14.97,-20.3428,57.7575
1,howard springs,AU,81.55,67.0,100.0,3.53,-12.4970,131.0470
2,port alfred,ZA,63.18,95.0,100.0,1.99,-33.5906,26.8910
3,iqaluit,CA,0.70,77.0,20.0,0.00,63.7506,-68.5145
4,busselton,AU,64.54,51.0,4.0,11.16,-33.6500,115.3333
...,...,...,...,...,...,...,...,...
624,onega,RU,34.07,82.0,100.0,8.86,63.9061,38.1404
626,xai-xai,MZ,78.28,68.0,3.0,6.78,-25.0519,33.6442
627,sabzevar,IR,83.48,20.0,40.0,16.11,36.2126,57.6819
628,vao,NC,74.89,75.0,79.0,20.31,-22.6667,167.4833


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)

In [19]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
target_city_df = weather_df.loc[(weather_df["Temperature"] > 70) & (weather_df["Temperature"] < 80) & 
                                (weather_df["Windspeed"] < 10) & (weather_df["Cloudiness"] == 0), :]
target_city_df = target_city_df.dropna(how='any')
target_city_df.reset_index(inplace=True)
del target_city_df['index']
target_city_df.head()

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude
0,port hedland,AU,78.85,65.0,0.0,8.05,-20.3167,118.5667
1,tura,IN,76.41,74.0,0.0,4.72,25.5198,90.2201
2,hamadan,IR,76.08,10.0,0.0,6.91,34.7992,48.5146


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = pd.DataFrame(target_city_df, columns=["City", "Latitude", "Longitude"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Latitude,Longitude,Hotel Name
0,port hedland,-20.3167,118.5667,
1,tura,25.5198,90.2201,
2,hamadan,34.7992,48.5146,


In [22]:
hotel_df = target_city_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: port hedland.
Closest hotel is The Esplanade Hotel.
------------
Retrieving Results for Index 1: tura.
Closest hotel is Hotel RIKMAN Continental.
------------
Retrieving Results for Index 2: hamadan.
Closest hotel is Parsian BuAli Hotel.
------------


In [23]:
hotel_df


,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Hotel Name
0,port hedland,AU,78.85,65.0,0.0,8.05,-20.3167,118.5667,The Esplanade Hotel
1,tura,IN,76.41,74.0,0.0,4.72,25.5198,90.2201,Hotel RIKMAN Continental
2,hamadan,IR,76.08,10.0,0.0,6.91,34.7992,48.5146,Parsian BuAli Hotel


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


# Display figure
